In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

used_cars = pd.read_csv('/kaggle/input/used-car-traders-dealers-in-pakistan/Used Car Trader  Dealers in Pakistan.csv')
pk_cities = pd.read_csv('/kaggle/input/pakistan-cities/pk.csv')
dial_codes = pd.read_csv('/kaggle/input/d/abdulazizniazi/pakistan-cities-dialcalling-code-source-ptcl/Pakistan Cities Calling Code.csv')

# Exploring
Understanding data and the requirements.

In [ ]:
used_cars.describe()

In [ ]:
used_cars.head()

In [ ]:
pk_cities

In [ ]:
dial_codes

Importing Dial Codes to cross match with addresses that doesn't have city name in address

# Cleaning Data

In [ ]:
def remove_city_suffix(row):
    if "City".lower() in row.city.lower():
        return row.city[:row.city.index(" City")]
    else:
        return row.city

Removing City suffix in the data we used for cities.

In [ ]:
pk_cities.city = pk_cities.apply(lambda row:remove_city_suffix(row), axis = 1)
pk_cities

In [ ]:
used_cars.isnull().sum()

In [ ]:
used_cars.Address.fillna("others")

> Some addresses are null, we can figure out there city by checking their phone number e.g if phone starts with 021 (21) the dealer is from karachi. However, if the number provided starts with 03XX then that will be a mobile number and we can't be sure about the address.For the above reason we will treat these dealerships as others for now

In [ ]:
unique_add = used_cars.Address
unique_add

I can see some addresses are just mobile numbers. We will treat these entries as "others".

# Filtering Cities
Finding city names in data in addresses and cross matching phone dial codes for unknown enteries.

In [ ]:
def city_names(row):
    address = row.Address
    cities = pk_cities.city
    phone = str(row.Phone)
    if not isinstance(address,str):
        address = str(address)
    for city in cities:
        if city.lower() in address.lower():
            return city
        
    for i,phone_row in dial_codes.iterrows():
        if phone.startswith(str(phone_row.area_codes)):
            for c in cities:
                if c.lower() in phone_row.city.lower():
                    return c
            return phone_row.city
    return "others"

In [ ]:
city_data = used_cars.apply(lambda row: city_names(row), axis = 1)

In [ ]:
result_data = pd.Series(city_data,name="City").to_frame().drop_duplicates(subset="City")
result_data["Count"] = result_data.apply(lambda row: city_data.value_counts().to_frame()[0][row.City], axis = 1)
result_data = result_data.sort_values("Count",ascending = False).reset_index(drop=True)
result_data

# Results

In [ ]:
arr = np.array(result_data.Count.tolist())
plt.pie(arr,labels =  result_data.City,autopct='%1.1f%%')
plt.rcParams.update({'font.size': 20,'figure.figsize':(26,26)})
plt.title("Cities with most used car dealerships")
plt.show() 
plt.savefig('Cities with most used car dealerships.png')

# Conclusion
> We can conclude that Karachi has most used cars dealerships followed by Lahore. However, provided data had lacked city names in addresses and/or dial codes which resulted in "others" as biggest part of the data.